## Dataquest - SQL Basics
This notebook has been done taking as a guide the following post: <br>
https://www.dataquest.io/blog/sql-basics/

### Libraries import

In [175]:
import sqlite3
import pandas as pd

### Database connection

In [176]:
db = sqlite3.connect("hubway.db")

Database file: https://www.dataquest.io/blog/images/jcoe-intro-to-sql/hubway.db

### Creating <i>run_query</i> function (with pandas output)

In [177]:
def run_query(query):
    return pd.read_sql_query(query, db)

### Queries

#### List of SQLite database objects

In [178]:
query = 'select * from sqlite_master;'
run_query(query)

,type,name,tbl_name,rootpage,sql
0,table,trips,trips,2,"CREATE TABLE trips (id INTEGER, duration INTEG..."
1,table,stations,stations,33340,"CREATE TABLE stations (id INTEGER, station TEX..."


Taking a look at the tables:

In [179]:
query = 'select * from trips limit 3;'
run_query(query)

,id,duration,start_date,start_station,end_date,end_station,bike_number,sub_type,zip_code,birth_date,gender
0,1,9,2011-07-28 10:12:00,23,2011-07-28 10:12:00,23,B00468,Registered,'97217,1976.0,Male
1,2,220,2011-07-28 10:21:00,23,2011-07-28 10:25:00,23,B00554,Registered,'02215,1966.0,Male
2,3,56,2011-07-28 10:33:00,23,2011-07-28 10:34:00,23,B00456,Registered,'02108,1943.0,Male


In [180]:
query = 'select * from stations limit 3;'
run_query(query)

,id,station,municipality,lat,lng
0,3,Colleges of the Fenway,Boston,42.340021,-71.100812
1,4,Tremont St. at Berkeley St.,Boston,42.345392,-71.069616
2,5,Northeastern U / North Parking Lot,Boston,42.341814,-71.090179


#### What was the duration of the longest trip?

In [181]:
query = '''
Select duration, start_date 
    from trips
    order by 1 desc
    limit 1;
'''
run_query(query)

,duration,start_date
0,9999,2012-09-26 18:34:00


#### How many trips were taken by ‘registered’ users?

In [182]:
query = '''
Select  count(*) "Trips by registered users"
    from trips
    where sub_type = "Registered";
'''
run_query(query)

,Trips by registered users
0,1105192


#### Do registered or casual users take longer trips?

In [183]:
query = '''
Select  sub_type, avg(duration) "Average duration"
    from trips
    group by sub_type;
'''
run_query(query)

,sub_type,Average duration
0,Casual,1519.643897
1,Registered,657.026067


#### Which bike was used for the most trips?
Bike B00490 with 2120 trips.

In [184]:
query = '''
Select  bike_number, count(*) Trips
    from trips
    group by bike_number
    order by Trips desc
    limit 5;
'''
run_query(query)

,bike_number,Trips
0,B00490,2120
1,B00268,2104
2,B00548,2099
3,B00559,2083
4,B00563,2067


#### Average duration of trips by registered members over the age of 30.

In [185]:
query = '''
Select avg(duration)
    from trips
    where (2017 - birth_date) > 30;
'''
run_query(query)

,avg(duration)
0,923.014685


#### Which station is the most frequent starting point?
While doing the exercise I thought about grouping by id and station name, and I noticed some differences regarding proposed query.


In [186]:
query = '''
select s.id, s.station, count(t.id) total_trips
    from trips t, stations s
    where t.start_station = s.id
    group by s.id, s.station
    order by total_trips desc
    limit 5;
'''
run_query(query)

,id,station,total_trips
0,22,South Station - 700 Atlantic Ave.,56123
1,36,Boston Public Library - 700 Boylston St.,41994
2,53,Beacon St / Mass Ave,35275
3,67,MIT at Mass Ave / Amherst St,33644
4,16,Back Bay / South End Station,32677


Proposed query by the tutorial:

In [187]:
query = '''
SELECT stations.station AS "Station", COUNT(*) AS "Count"
FROM trips 
JOIN stations
ON trips.start_station = stations.id
GROUP BY stations.station
ORDER BY COUNT(*) DESC
LIMIT 5;
'''

run_query(query)

,Station,Count
0,South Station - 700 Atlantic Ave.,56123
1,Boston Public Library - 700 Boylston St.,41994
2,Charles Circle - Charles St. at Cambridge St.,35984
3,Beacon St / Mass Ave,35275
4,MIT at Mass Ave / Amherst St,33644


It shows <i>Charles Circle - Charles St. at Cambridge St</i> as the 3rd position having more starting trips (35984) than my 3rd result (<i>Beacon St / Mass Ave</i> with 35275).<br>
That's due to the duplicated stations (perhaps bad data quality):

In [188]:
query = '''
select * from stations 
where station in (
    select station from stations
        group by station
        having count(*) > 1)
order by station, id;
'''
run_query(query)

,id,station,municipality,lat,lng
0,85,Andrew Station - Dorchester Ave at Humboldt Pl,Boston,42.330825,-71.057007
1,104,Andrew Station - Dorchester Ave at Humboldt Pl,Boston,42.330715999999995,-71.05704399999999
2,61,Boylston at Fairfield,Boston,42.348323,-71.082674
3,103,Boylston at Fairfield,Boston,42.348762,-71.082383
4,60,Charles Circle - Charles St. at Cambridge St.,Boston,42.360877,-71.07131
5,113,Charles Circle - Charles St. at Cambridge St.,Boston,42.360690000000005,-71.07088399999999
6,56,Dudley Square,Boston,42.3281898,-71.0833545
7,100,Dudley Square,Boston,42.328654,-71.084198
8,23,Mayor Thomas M. Menino - Government Center,Boston,42.359677000000005,-71.059364
9,99,Mayor Thomas M. Menino - Government Center,Boston,42.35892,-71.057629


#### Which stations are most frequently used for round trips?



In [189]:
query = '''
select s.station, count(t.id) total_trips
    from trips t, stations s
    where t.start_station = s.id
    and t.start_station = t.end_station
    group by s.station
    order by total_trips desc
    limit 5;
'''
run_query(query)

,station,total_trips
0,The Esplanade - Beacon St. at Arlington St.,3064
1,Charles Circle - Charles St. at Cambridge St.,2739
2,Boston Public Library - 700 Boylston St.,2548
3,Boylston St. at Arlington St.,2163
4,Beacon St / Mass Ave,2144


#### How many trips start and end in different municipalities?



In [190]:
import timeit

query = '''
select count(*)
    from trips, stations start, stations end
    where start_station = start.id --join start stations names
    and end_station = end.id --join end stations names
    and start.municipality <> end.municipality;
'''
tic = timeit.default_timer()
run_query(query)
toc = timeit.default_timer()
print("Time:", toc - tic) #seconds

run_query(query)

Time: 0.9177719693652762


,count(*)
0,309748


Another way to get the same result is by doing 2 subqueries: one for start trips and another for end trips.<br>But, as it can be seen taking the time, it is almost 3 times slower than with the previous query.

In [191]:
import timeit

query = '''
select count(*) from 
(select t.id, t.start_station, s.station, s.municipality
    from trips t, stations s
    where t.start_station = s.id) START, 
(select t.id, t.end_station, s.station, s.municipality
    from trips t, stations s
    where t.end_station = s.id) END
where START.id = END.id --join trips IDs from different subqueries
and START.municipality <> END.municipality;
'''
tic = timeit.default_timer()
run_query(query)
toc = timeit.default_timer()
print("Time:", toc - tic) #seconds

run_query(query)

Time: 2.6816677148053714


,count(*)
0,309748


#### How many trips incurred additional fees (lasted longer than 30 minutes)?

123155 trips lasted longer than 30 minutes.

In [192]:
query = '''
select count(*)
    from trips
    where duration > (60*30);
'''
run_query(query)

,count(*)
0,123155


#### Which bike was used for the longest total time?
Bike number B00490.



In [193]:
query = '''
select bike_number, sum(duration) total_duration
    from trips
    group by bike_number
    order by total_duration desc
    limit 5;
'''
run_query(query)

,bike_number,total_duration
0,B00490,2058643
1,B00548,1996532
2,B00329,1986397
3,B00268,1969755
4,B00355,1965279


#### Did registered or casual users take more round trips?
Casual users take more round trips.


In [194]:
query = '''
select sub_type, count(*)
    from trips t, stations s
    where t.start_station = s.id
    and t.start_station = t.end_station
    group by sub_type
    order by count(*) desc;
'''
run_query(query)

,sub_type,count(*)
0,Casual,41427
1,Registered,31635


#### Which municipality had the longest average duration?
Brookline has the longest average duration (1020.84 seconds for start stations and 1069.86 seconds for end stations).

In [195]:
query = '''
select municipality, avg(duration) avg_duration
    from trips t, stations s
    where t.start_station = s.id
    --where t.end_station = s.id
    group by municipality
    order by 2 desc;
'''
run_query(query)

,municipality,avg_duration
0,Brookline,1020.844836
1,Boston,929.211749
2,Somerville,866.462890
3,Cambridge,842.171816
